In [1]:
%load_ext autoreload
import os
import sys

os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ["OPENAI_API_KEY"]= 'xxx'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('./src-py')

In [2]:
import json
keys = json.load(open('/mnt/swordfish-pool2/milad/code/llms-as-science-communicators/keys.json'))
for key, val in keys.items():
    os.environ[key] = val

In [3]:
import datasets
import json
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


%autoreload
import utils
import datadreamer_generation

import tiktoken
from transformers import AutoTokenizer

/mnt/swordfish-pool2/milad/conda-envs/trl-library-env/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


ModuleNotFoundError: No module named 'datadreamer'

In [ ]:
from huggingface_hub import login

login(keys['hf_token'])

In [ ]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/'

In [ ]:
gpt_tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

### Generate the final conversation dataset

In [7]:
training_ds = datasets.load_from_disk(output_dir + "/high_quality_news_dataset")

In [8]:
len(training_ds)

19812

In [9]:
all_prompts = utils.get_prompt_compositions()
used_prompt = all_prompts[2]

In [10]:
used_prompt = all_prompts[2]

In [11]:
# used_prompt['instruction'] = """
# Please extract the main questions answered in the [JOURNALISTIC-REPORT]. 
# Then, simulate a conversation between a journalist and researcher where the journalist asks the extracted main questions and for
# For every researcher's answer, the journalist must ask a follow-up question to clarify unclear aspects

# """
# used_prompt['inputs'] = {'[SCIENTIFIC-PAPER]': 'sc-intro', '[JOURNALISTIC-REPORT]': 'pr-article'}

used_prompt['instruction'] = """
Please extract all questions answered in the [JOURNALISTIC-REPORT]. 
Then, simulate a conversation between a journalist and researcher where the journalist asks the extracted questions.
For every researcher's answer, the journalist must ask a follow-up question to clarify unclear aspects.
"""
used_prompt['inputs'] = {'[SCIENTIFIC-PAPER]': 'sc-intro', '[JOURNALISTIC-REPORT]': 'pr-article'}

In [12]:
deepseek_tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-14B', trust_remote_code=True)

In [13]:
#output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-updated/'
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-32-final-conv-ds/'
resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'deepseek-r1', 
                                                           training_ds, used_prompt, deepseek_tokenizer, 
                                                           max_input_tokens=5000, max_new_tokens=1250, tmp=0.6, top_p=0.95)
resulted_ds.save_to_disk(output_dir)

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-32-final-conv-ds/composite-na-pr-guided
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and the results were backed up. 💾
[ 🤖 DataDreamer 💤 ] Step 'documents' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
/mnt/swordfish-pool2/milad/code/llms-as-science-communicators/src-py/datadreamer_generation.py:81: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_truncated': truncate_text(encoding, row['inputs'], max_input_tok

DatabaseError: file is not a database

In [21]:
resulted_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_truncated', 'conversation', 'parsed_conv'],
    num_rows: 1000
})

In [ ]:
print(resulted_ds['conversation'][0])

In [ ]:
# output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/llama3-final-conv-ds/'
# resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'llama3', training_ds, used_prompt, llama_tokenizer, max_input_tokens=5000)
# resulted_ds.save_to_disk(output_dir)

In [104]:
# output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/gpt3-final-conv-ds/'
# resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'gpt-3', training_ds, used_prompt, gpt_tokenizer)
# resulted_ds.save_to_disk(output_dir)

In [ ]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds/'
resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'deepseek-r1', training_ds, used_prompt, gpt_tokenizer, max_input_tokens=5000)
resulted_ds.save_to_disk(output_dir)

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds/composite-na-pr-guided
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'documents' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
/local/nlp/milad/code/communicating-science-to-the-public/src/datadreamer_generation.py:159: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_len': len(encoding.encode(row['inputs']))})
[ 🤖 DataDreamer 💤 ] Step 'documents (map) (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documen

### Cleaning up the corpus:

In [211]:
#deepseek_conv_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds')
deepseek_conv_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-updated/')

In [212]:
deepseek_conv_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_truncated', 'conversation', 'parsed_conv'],
    num_rows: 19812
})

In [213]:
deepseek_conv_ds = deepseek_conv_ds.filter(lambda row: '</think>' in row['conversation'])
deepseek_conv_ds = deepseek_conv_ds.map(lambda row: utils.parse_conversation(row))
deepseek_conv_ds = deepseek_conv_ds.filter(lambda row: len(row['parsed_conv'])>0)

Map:   0%|          | 0/19738 [00:00<?, ? examples/s]

Error parsing


### Main Questions Answered in the [JOURNALISTIC-REPORT]:

1. **What challenges have hindered brain research due to the brain's structure?**
   - The thick, hard skull makes accessing the brain difficult, limiting researchers to low-resolution imaging or external signal measurements.

2. **How have artificial brain models been used in research?**
   - Artificial brain models created from primary neurons or iPSCs have been used to study brain development and disease causes.

3. **What limitations did previous 2D and early 3D brain models face?**
   - 2D models didn't replicate real brain structures, and early 3D models lacked an analytical framework, limiting signal analysis to surface measurements or flattened structures.

4. **What is the significance of the 3D multifunctional system developed by KIST?**
   - It allows precise, non-destructive stimulation and real-time measurement of neural signals within a 3D artificial brain model.

5. **What capabilities does the 3D

Filter:   0%|          | 0/19738 [00:00<?, ? examples/s]

In [214]:
deepseek_conv_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_truncated', 'conversation', 'parsed_conv'],
    num_rows: 19474
})

In [216]:
print(deepseek_conv_ds['parsed_conv'][8])

[{'author': 'Journalist', 'text': ' Dr. Auteri, your study found genetic evidence of resistance in some bats to white-nose syndrome. What specific evidence did you uncover, and how does this contribute to our understanding of the disease?'}, {'author': 'Researcher', 'text': ' Thank you for asking. We identified significant differences in three key genes'}, {'author': 'Journalist', 'text': ' That’s fascinating. How did you identify these genetic differences between survivors and non-survivors? Did you use a specific method or technology?'}, {'author': 'Researcher', 'text': ' We conducted a genome-wide scan using ddRadSeq, which allowed us to compare 19,797 SNPs across 14,345 loci from 25 non-survivors and 9 survivors. We applied F ST-outlier analyses to detect loci under selection, which highlighted the nine SNPs associated with survival. This approach helped us distinguish between adaptive changes driven by selection and non-adaptive changes due to genetic drift.'}, {'author': 'Journal

In [217]:
deepseek_conv_ds.save_to_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-cleaned')
#deepseek_conv_ds.save_to_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned')

Saving the dataset (0/5 shards):   0%|          | 0/19474 [00:00<?, ? examples/s]

In [220]:
deepseek_conv_ds.push_to_hub("miladalsh/new-deepseek-final-conv-ds-cleaned")
#deepseek_conv_ds.push_to_hub("miladalsh/deepseek-final-conv-ds-cleaned")

Uploading the dataset shards:   0%|          | 0/5 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-cleaned/commit/b18e2b84a29a5a3c197bfb705d2a5f0e5b85a1d5', commit_message='Upload dataset', commit_description='', oid='b18e2b84a29a5a3c197bfb705d2a5f0e5b85a1d5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-cleaned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='miladalsh/new-deepseek-final-conv-ds-cleaned'), pr_revision=None, pr_num=None)

In [221]:
deepseek_conv_ds['Topic'][:10]

['Medicine',
 'Medicine',
 'Medicine',
 'Nano',
 'Medicine',
 'Other',
 'Earth',
 'Medicine',
 'Biology',
 'Medicine']

### Cleaning up the corpus:

In [17]:
#deepseek_conv_ds = datasets.load_dataset("miladalsh/deepseek-final-conv-ds-clead")
deepseek_conv_ds = datasets.load_dataset("miladalsh/new-deepseek-final-conv-ds-cleaned")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [18]:
deepseek_conv_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_truncated', 'conversation', 'parsed_conv'],
        num_rows: 19474
    })
})

In [19]:
journalist_questions = [row['id'] + ' ==> ' + item['text'].strip() for row in deepseek_conv_ds['train'] for item in row['parsed_conv'] if item['author'] == 'Journalist']

In [20]:
journalist_questions[:300]

['10.1038/s41586-019-1004-y ==> Dr. Beatty, the study reveals that hepatocytes are central to creating a pro-metastatic niche in the liver. Can you explain why the liver is the most common site for cancer metastasis?',
 "10.1038/s41586-019-1004-y ==> So, the liver's own cells are essentially creating a nurturing environment for cancer cells. That's fascinating. How exactly does the IL-6–STAT3–SAA pathway contribute to this process?",
 '10.1038/s41586-019-1004-y ==> If IL-6 is so critical, does this mean that blocking IL-6 could be a potential therapeutic strategy to prevent liver metastasis?',
 '10.1038/s41586-019-1004-y ==> The study also mentions that elevated levels of circulating SAA correlate with worse outcomes in patients. Can you explain the clinical implications of these findings?',
 "10.1038/s41586-019-1004-y ==> Finally, the study points out that conditions associated with increased IL-6 levels, such as obesity or cardiovascular disease, could influence the liver's receptive

In [16]:
journalist_questions[:300]

['10.1038/s41586-019-1004-y ==> Dr. Beatty, your study published in *Nature* reveals that hepatocytes play a central role in creating a pro-metastatic niche in the liver. Can you explain how this discovery changes our understanding of how cancer spreads to the liver?',
 '10.1038/s41586-019-1004-y ==> The study mentions that blocking IL-6 can limit liver metastasis. How significant is this finding, and what potential therapies could this lead to?',
 '10.1038/s41586-019-1004-y ==> The study also highlights the importance of the liver as a sensor of inflammation. How does this insight help us understand the broader implications of chronic inflammation in cancer progression?',
 '10.1038/s41586-019-1004-y ==> In the study, you used mouse models of pancreatic ductal adenocarcinoma. How confident are you that these findings translate to human patients, and what steps are being taken to validate this in clinical settings?',
 '10.1038/s41586-019-1004-y ==> The study identifies hepatocytes as th

In [236]:
len(journalist_questions)

126779

In [238]:
journalist_questions[:300]

['10.1038/s41586-019-1004-y ==> Dr. Beatty, the study reveals that hepatocytes are central to creating a pro-metastatic niche in the liver. Can you explain why the liver is the most common site for cancer metastasis?',
 "10.1038/s41586-019-1004-y ==> So, the liver's own cells are essentially creating a nurturing environment for cancer cells. That's fascinating. How exactly does the IL-6–STAT3–SAA pathway contribute to this process?",
 '10.1038/s41586-019-1004-y ==> If IL-6 is so critical, does this mean that blocking IL-6 could be a potential therapeutic strategy to prevent liver metastasis?',
 '10.1038/s41586-019-1004-y ==> The study also mentions that elevated levels of circulating SAA correlate with worse outcomes in patients. Can you explain the clinical implications of these findings?',
 "10.1038/s41586-019-1004-y ==> Finally, the study points out that conditions associated with increased IL-6 levels, such as obesity or cardiovascular disease, could influence the liver's receptive

In [20]:
processed_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-preprocessed-for-llama3/train_journalist_ds/')
processed_ds.push_to_hub('miladalsh/new-deepseek-final-conv-ds-cleaned-and-processed')

Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/49.4M [00:00<?, ?B/s]

...

Uploading...:   0%|          | 0.00/49.4M [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-cleaned-and-processed/commit/b74bea1f85b6d46f1351bec828087b368c60bc67', commit_message='Upload dataset', commit_description='', oid='b74bea1f85b6d46f1351bec828087b368c60bc67', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-cleaned-and-processed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='miladalsh/new-deepseek-final-conv-ds-cleaned-and-processed'), pr_revision=None, pr_num=None)

## Preference Dataset

In [17]:
def print_sample(sample):
    print(sample['prompt'][-1]['content'], '\n\n')
    print(sample['answer_quality']['complex_aspects'], sample['answer_quality']['is_vague'], '\n\n')
    print(sample['chosen'], ' >>> ',ds[10]['rejected'], '\n\n')

In [12]:
ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-with-preference-data-20k/')
qwen_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-with-preference-data-20k-for-qwen/')

In [13]:
is_vague = [row['answer_quality']['is_vague'] for  row in ds]
complex_aspects = [len(row['answer_quality']['complex_aspects']) > 0 for  row in ds]
print(is_vague.count(0)/len(is_vague), complex_aspects.count(True)/len(complex_aspects))

is_vague = [row['answer_quality']['is_vague'] for  row in qwen_ds]
complex_aspects = [len(row['answer_quality']['complex_aspects']) > 0 for  row in qwen_ds]
print(is_vague.count(0)/len(is_vague), complex_aspects.count(True)/len(complex_aspects))

0.9369359970789213 0.5616817067445621
0.28148959474260676 0.8320033380274344


In [25]:
len(complex_aspects)

19173

In [20]:
print_sample(ds[5])

Researcher:   
We found that all non-African populations experienced a significant population bottleneck between 50,000 and 70,000 years ago. For example, European populations, including those from the CEU, GBR, and TSI populations, showed a bottleneck around this time. However, the Finnish population (FIN) had a distinct bottleneck between 10,000 and 20,000 years ago, suggesting a separate event related to their isolation from the rest of Europe. Additionally, we observed a bottleneck in African populations around 50,000 to 70,000 years ago and another around 20,000 to 30,000 years ago. 


['population bottleneck', 'CEU', 'GBR', 'TSI', 'FIN'] 0 


Here is the follow-up question:

Could you clarify why the stairway plot method specifically highlights the Finnish population as having a distinct bottleneck between 10,000 and 20,000 years ago, and what evidence supports this conclusion?  >>>  What are the specific benefits of using the SynNotch gated CAR-T cells compared to traditional CA

In [21]:
print_sample(qwen_ds[5])

Researcher:   
We found that all non-African populations experienced a significant population bottleneck between 50,000 and 70,000 years ago. For example, European populations, including those from the CEU, GBR, and TSI populations, showed a bottleneck around this time. However, the Finnish population (FIN) had a distinct bottleneck between 10,000 and 20,000 years ago, suggesting a separate event related to their isolation from the rest of Europe. Additionally, we observed a bottleneck in African populations around 50,000 to 70,000 years ago and another around 20,000 to 30,000 years ago. 


['population bottleneck', 'CEU', 'GBR', 'TSI', 'FIN'] 1 


Follow-up Question: Could you clarify why the Finnish population experienced a different bottleneck compared to other European populations, and what factors might have contributed to this unique event?  >>>  What are the specific benefits of using the SynNotch gated CAR-T cells compared to traditional CAR-T cells in treating neuroblastoma? 
